---
title: Migrating from RefineDocumentsChain
---

[RefineDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.refine.RefineDocumentsChain.html) implements a strategy for analyzing long texts. The strategy is as follows:

- Split a text into smaller documents;
- Apply a process to the first document;
- Refine or update the result based on the next document;
- Repeat through the sequence of documents until finished.

A common process applied in this context is summarization, in which a running summary is modified as we proceed through chunks of a long text. This is particularly useful for texts that are large compared to the context window of a given LLM.

An [LCEL](/docs/concepts/#langchain-expression-language-lcel) implementation confers a number of advantages for this problem, including support for token-by-token streaming. Because it is assembled from modular components, it is also simple to extend or modify (e.g., to incorporate [tool calling](/docs/concepts/#functiontool-calling) or other behavior).

Below we will go through both `RefineDocumentsChain` and a corresponding LCEL implementation on a simple example for illustrative purposes.

Let's first load a chat model:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [1]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

## Example

Let's go through an example where we summarize a sequence of documents. We first generate some simple documents for illustrative purposes:

In [3]:
from langchain_core.documents import Document

documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book"}),
]

### Legacy

Below we show an implementation with `RefineDocumentsChain`. We define the prompt templates for the initial summarization and successive refinements, instantiate separate [LLMChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html) objects for these two purposes, and instantiate `RefineDocumentsChain` with these components.

In [5]:
from langchain.chains import LLMChain, RefineDocumentsChain
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

# This controls how each document will be formatted. Specifically,
# it will be passed to `format_document` - see that function for more
# details.
document_prompt = PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
)
document_variable_name = "context"
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
# The prompt here should take as an input variable the
# `document_variable_name`
summarize_prompt = ChatPromptTemplate(
    [
        ("human", "Write a concise summary of the following: {context}"),
    ]
)
initial_llm_chain = LLMChain(llm=llm, prompt=summarize_prompt)
initial_response_name = "existing_answer"
# The prompt here should take as an input variable the
# `document_variable_name` as well as `initial_response_name`
refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_llm_chain = LLMChain(llm=llm, prompt=refine_prompt)
chain = RefineDocumentsChain(
    initial_llm_chain=initial_llm_chain,
    refine_llm_chain=refine_llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    initial_response_name=initial_response_name,
)

We can now invoke our chain:

In [6]:
result = chain.invoke(documents)
result["output_text"]

'Apples are red, blueberries are blue, and bananas are yellow in color.'

The [LangSmith trace](https://smith.langchain.com/public/8ec51479-9420-412f-bb21-cb8c9f59dfde/r) is composed of three LLM calls: one for the initial summary, and two more updates of that summary. The process completes when we update the summary with content from the final document.

### LCEL

Below we show an LCEL implementation of this process:

- We use the same two templates as before.
- We generate a simple chain for the initial summary that plucks out the first document, formats it into a prompt and runs inference with our LLM.
- We generate a second `_refine_remaining_docs` chain that generates a sequence of simple `prompt | llm | StrOutputParser()` chains, formatting the context from each remaining document into the prompt.

In [7]:
from typing import List

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
)
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

summarize_prompt = ChatPromptTemplate(
    [
        ("human", "Write a concise summary of the following: {context}"),
    ]
)

initial_summary = (
    RunnableLambda(lambda docs: docs[0].page_content)  # select first document
    | summarize_prompt  # Format content into summarize prompt
    | llm  # Generate response
    | StrOutputParser()  # Cast to string
)

refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])


def _refine_remaining_docs(input_dict: dict) -> RunnableSequence:
    """Create a sequence of summarizers, each adding context from a document."""
    steps = []
    for document in input_dict["documents"]:
        steps.append(
            # format text from document[i] into refine prompt
            refine_prompt.partial(context=document.page_content)
            | llm  # generate response
            | StrOutputParser()  # cast to string
        )
    return RunnableSequence(*steps)


chain = (
    {
        # create initial summary
        "existing_answer": initial_summary,
        # pass along remaining documents
        "documents": RunnableLambda(lambda docs: docs[1:]),
    }
    # create and run a sequence of summaries on remaining docs
    | RunnableLambda(_refine_remaining_docs)
)

We can invoke the chain as before:

In [9]:
chain.invoke(documents)

'Apples are typically red in color, blueberries are blue, and bananas are yellow.'

In the [LangSmith trace](https://smith.langchain.com/public/6b038c7a-c56b-4ae2-9d82-686020146c43/r) we again recover three LLM calls, performing the same functions as before.

Note that the LCEL implementation allows us to stream output tokens, if desired:

In [10]:
for chunk in chain.stream(documents):
    print(chunk, end=" | ")

 | App | les |  are |  a |  red |  fruit | , |  blue | berries |  are |  blue | , |  and |  bananas |  are |  yellow | . |  | 

## Next steps

Check out the [LCEL conceptual docs](/docs/concepts/#langchain-expression-language-lcel) for more background information.

See [this tutorial](/docs/tutorials/summarization/) for more LLM-based summarization strategies.